선수 연도별 성적 크롤링

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

name_list = ["송성문", "김혜성", "이주형", "최주환", "김건희", "변상권", "김태진", "도슨", "이용규"]
code_list = ["11376", "12905", "14591", "10182", "15472", "13223", "11244", "15652", "10035"]

for i in range(len(name_list)):
    # HTML 데이터 가져오기
    url = f"https://statiz.sporki.com/player/?m=year&p_no={code_list[i]}"  # 실제 URL로 변경
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # 데이터 추출
    table = soup.find("table")
    rows = table.find("tbody").find_all("tr")

    data = []
    columns = ["year", "h_avg", "obp", "slg", "ops", "wrc_plus", "h", "2b", "3b", "hr", "rbi", "sb", "bb", "so", "pa", "ab", "war"]

    for row in rows:
        cells = row.find_all("td")
        if len(cells) > 0:
            year = cells[0].text.strip()
            
            # 필요한 값들을 문자열에서 실수로 변환
            avg = float(cells[26].text.strip()) if cells[26].text.strip() else 0.0
            obp = float(cells[27].text.strip()) if cells[27].text.strip() else 0.0
            slg = float(cells[28].text.strip()) if cells[28].text.strip() else 0.0
            ops = float(cells[29].text.strip()) if cells[29].text.strip() else 0.0
            wrc_plus = float(cells[31].text.strip()) if cells[31].text.strip() else 0.0
            h = int(cells[11].text.strip()) if cells[11].text.strip() else 0
            two_b = int(cells[12].text.strip()) if cells[12].text.strip() else 0
            three_b = int(cells[13].text.strip()) if cells[13].text.strip() else 0
            hr = int(cells[14].text.strip()) if cells[14].text.strip() else 0
            rbi = int(cells[16].text.strip()) if cells[16].text.strip() else 0
            sb = int(cells[17].text.strip()) if cells[17].text.strip() else 0
            bb = int(cells[19].text.strip()) if cells[19].text.strip() else 0
            hp = int(cells[20].text.strip()) if cells[20].text.strip() else 0
            ib = int(cells[21].text.strip()) if cells[21].text.strip() else 0
            so = int(cells[22].text.strip()) if cells[22].text.strip() else 0
            pa = int(cells[7].text.strip()) if cells[7].text.strip() else 0
            ab = int(cells[9].text.strip()) if cells[9].text.strip() else 0
            war = float(cells[-1].text.strip()) if cells[-1].text.strip() else 0.0

            # bb, hp, ib를 더하고 bb로만 남기기
            bb = bb + hp + ib

            # 데이터 추가
            data.append([year, avg, obp, slg, ops, wrc_plus, h, two_b, three_b, hr, rbi, sb, bb, so, pa, ab, war])

    # DataFrame 생성
    df = pd.DataFrame(data, columns=columns)

    # CSV 저장
    df.to_csv(f"{name_list[i]}_stats.csv", index=False)


선수 상세정보 크롤링

In [37]:
name_list = ["송성문", "김혜성", "이주형", "최주환", "김건희", "변상권", "김태진", "도슨", "이용규", "후라도", "헤이수스", "하영민", "주승우"]
code_list = ["11376", "12905", "14591", "10182", "15472", "13223", "11244", "15652", "10035", "15531", "16138", "11222", "15067"]

In [40]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import pandas as pd

def convert_date(date_str):
    # '1996년 08월 29일' 형식에서 숫자만 추출
    match = re.match(r'(\d{4})년 (\d{2})월 (\d{2})일', date_str)
    if match:
        year = match.group(1)
        month = match.group(2)
        day = match.group(3)
        
        # 'YYYY-MM-DD' 형식으로 변환 (datetime 사용)
        date_obj = datetime(year=int(year), month=int(month), day=int(day))
        return date_obj.strftime('%Y-%m-%d')  # SQL에 적합한 형식으로 반환
    else:
        raise ValueError("Invalid date format")


def convert_pos_num(pos):
    pos_num = {"P":1 , "C":2, "1B":"3", "2B":"4", "3B":"5", "SS":"6", "LF":"7", "CF":"8", "RF":"9", "DH":"10"}
    return pos_num[pos]

def convert_Batting_Throw(info):
    batt_dict = {'좌타':False, '우타':True}
    throw_dict = {'좌투':False, '우투':True}
    return throw_dict[info[0:2]], batt_dict[info[2:4]]

# DataFrame columns 정의
columns = ["playerName", "playerBorn", "playerDraft", "playerPos", "playerBattingSide", "playerThrowSide"]
df = pd.DataFrame(columns=columns)


for i in range(len(code_list)):
    # 선수 정보 페이지 URL
    url = f'https://statiz.sporki.com/player/?m=playerinfo&p_no={code_list[i]}'

    # 요청을 보내고 페이지 내용 얻기
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 이름 추출
    name = soup.find('div', class_='name').text.strip()

    # 포지션, 팀, 타자/투수 정보 추출
    position_info = [span.text.strip() for span in soup.find('div', class_='con').find_all('span')]

    # 상세 정보 (생년월일, 출신학교, 신인지명, 활약연도, 활약팀) 추출
    man_info = {}
    for li in soup.find_all('li'):
        label = li.find('span')  # <span> 태그가 있으면
        if label:
            label_text = label.text.strip().replace(":", "")  # ':' 제거
            value_text = li.get_text().replace(label_text, "").strip()  # label_text는 이미 추출했으니 그것을 제거
            man_info[label_text] = value_text

    # 타자/투수 정보 변환
    throw, batt = convert_Batting_Throw(position_info[2])

    # 새로운 데이터 생성
    new_data = {
        "playerName": name_list[i],
        "playerBorn": convert_date(man_info.get('생년월일')),
        "playerDraft": man_info.get('신인지명', '정보 없음'),
        "playerPos": convert_pos_num(position_info[1]),
        "playerBattingSide": batt,
        "playerThrowSide": throw
    }

    # 새로운 행을 DataFrame에 추가
    new_df = pd.DataFrame([new_data])
    df = pd.concat([df, new_df], ignore_index=True)

# DataFrame을 CSV 파일로 저장
df.to_csv("선수_정보.csv", index=False)

print(df.tail())  # 첫 5행 출력 확인


   playerName  playerBorn         playerDraft playerPos playerBattingSide  \
8         이용규  1985-08-26  04 LG 2차 2라운드 15순위         7             False   
9         후라도  1996-01-30          23 키움 자유선발         1              True   
10       헤이수스  1996-12-10          24 키움 자유선발         1             False   
11        하영민  1995-05-07   14 넥센 2차 1라운드 4순위         1              True   
12        주승우  2000-01-30            22 키움 1차         1              True   

   playerThrowSide  
8            False  
9             True  
10           False  
11            True  
12            True  
